In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="2944a580-6c5f-4258-8c86-b9c5de957998"
RESOURCE_GROUP="calebebraga08-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

eastus2 : calebebraga08-rg


In [3]:
TRAIN_DATAS = [
    "vrex_1999_2000_2001_2002_2003_.csv",
    "vrex_2004_2005_2006_2007_2008_.csv",
    "vrex_2009_2010_2011_2012_2013_.csv",
    "vrex_2014_2015_2016_2017_2018_.csv",
]

TEST_DATAS = [
    "vrex_2004.csv",
    "vrex_2009.csv",
    "vrex_2014.csv",
    "vrex_2019.csv"
]

version = "original"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/bb48172e3ccf65075606876bcf5df9f2/vrex_1999_2000_2001_2002_2003_.csv - name: vrex_1999_2000_2001_2002_2003_
Data to test asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/3b205fc48a24468cd8575de6f0de7ae0/vrex_2004.csv - name: vrex_2004
Data to train asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/992a66ec6cfe9db64c4d0613ce5b5208/vrex_2004_2005_2006_2007_2008_.csv - name: vrex_2004_2005_2006_2007_2008_
Data to test asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments

# Load enviroment depedencies

In [4]:
ENV_VERSION = "test"

In [5]:
from azure.ai.ml.entities import Environment

custom_env_name = "data-prep-dependencies"

data_prep_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for data preparation",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_data_prep.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version=ENV_VERSION,
)
data_prep_job_env = ml_client.environments.create_or_update(data_prep_job_env)

print(
    f"Environment with name {data_prep_job_env.name} is registered to workspace, the environment version is {data_prep_job_env.version}"
)

Environment with name data-prep-dependencies is registered to workspace, the environment version is test


In [6]:
from azure.ai.ml.entities import Environment

custom_env_name = "train-dependencies"

train_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for train data",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_train.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version=ENV_VERSION,
)
train_job_env = ml_client.environments.create_or_update(train_job_env)

print(
    f"Environment with name {train_job_env.name} is registered to workspace, the environment version is {train_job_env.version}"
)

Environment with name train-dependencies is registered to workspace, the environment version is test


# Load components

In [7]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/data_prep.yaml")

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Uploading components (0.01 MBs): 100%|██████████| 8544/8544 [00:00<00:00, 71449.77it/s]




Component data_prep_vrex_defaults_model with Version 2024-03-26-14-44-38-4182449 is registered


In [8]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/train.yaml")

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Component train_vrex_defaults_model with Version 2024-03-26-14-44-39-9021762 is registered


# Pipeline

In [9]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def prep_data_and_train_defaults_pipeline(
    data_to_train,
    data_to_test,
    learning_rate_to_train,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
    )

    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=learning_rate_to_train,  # note: using a pipeline input as parameter
    )


In [10]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):
    pipeline = prep_data_and_train_defaults_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        learning_rate_to_train=0.05
    )
    
    pipelines.append(pipeline)

In [19]:
import datetime as dt

def _get_experiment_names() -> [str]:
    experiment_names = []
    for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
        current_time = dt.datetime.now()
        formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")  # Formata a data e hora atual
        train_name_base = train_name.split('.')[0]
        test_name_base = test_name.split('.')[0]
        name = f"{train_name_base}tested_{test_name_base}_executed_in_{formatted_time}"
        experiment_names.append(name)
        print(name)
    return experiment_names


In [20]:
experiment_names = _get_experiment_names()
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )

    ml_client.jobs.stream(pipeline_job.name)

vrex_1999_2000_2001_2002_2003_tested_vrex_2004_executed_in_2024_03_26_14_51_18
vrex_2004_2005_2006_2007_2008_tested_vrex_2009_executed_in_2024_03_26_14_51_18
vrex_2009_2010_2011_2012_2013_tested_vrex_2014_executed_in_2024_03_26_14_51_18
vrex_2014_2015_2016_2017_2018_tested_vrex_2019_executed_in_2024_03_26_14_51_18
RunId: willing_giraffe_1l66ll3bl5
Web View: https://ml.azure.com/runs/willing_giraffe_1l66ll3bl5?wsid=/subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments

Streaming logs/azureml/executionlogs.txt

[2024-03-26 14:51:22Z] Submitting 1 runs, first five are: d4924529:b0501f81-3cb3-47af-9438-a3f23b444c09
[2024-03-26 14:55:45Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: willing_giraffe_1l66ll3bl5
Web View: https://ml.azure.com/runs/willing_giraffe_1l66ll3bl5?wsid=/subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tc

JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /data_prep_job. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2024-03-26T14:55:45.393971Z",
    "component_name": ""
} 